<a href="https://colab.research.google.com/github/anscch/ATPAD-UNAM/blob/main/POLLUTION_ATPAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***POLLUTION_ATPAD***


# DATASET CONSTRUCTION

Parquet files contains all data from each station. This files will need to be updated as new inforatin is available. To update such files.....

First cleaning steps are carried out by concatenating all station's files into a single sataset since all parameters are the same between station. For further analysis such as outlier identification, data must be splited by region.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import duckdb
connection = duckdb.connect()
import os
import datetime

mor_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_MORE_1h_CAire_2024.parquet').reset_index(drop=True)
sal_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_SLLO_1h_CAire_2024.parquet').reset_index(drop=True)
agu_data_raw = pd.read_parquet('/content/drive/MyDrive/ATPAD_COLAB/PARQUET_FILES/RUOA_AGSC_1h_CAire_2024.parquet').reset_index(drop=True)

#filling gaps in timestamp

dfs = [mor_data_raw, sal_data_raw, agu_data_raw]

for i in range(3):
    full_range = pd.date_range(start=dfs[i]['Time'].min(), end=dfs[i]['Time'].max(), freq='h')
    dfs[i]= dfs[i].set_index('Time')
    dfs[i]= dfs[i].reindex(full_range)

dfs[0]['region'] = 'Morelia'
dfs[1]['region'] = 'Saltillo'
dfs[2]['region'] = 'Aguascalientes'

pollution_raw = pd.concat(dfs).reset_index(drop=False)
pollution_raw = pollution_raw.rename(columns={'index': 'Time'})

pollution_raw

,Time,O3,SO2,NO2,NO,CO,PM10,PM25,region
0,2015-08-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
1,2015-08-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
2,2015-08-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
3,2015-08-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
4,2015-08-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Morelia
...,...,...,...,...,...,...,...,...,...
197407,2018-10-15 23:00:00,9.85,0.54,9.86,7.37,NaN,9.32,NaN,Aguascalientes
197408,2018-10-16 00:00:00,7.70,0.53,9.75,7.33,NaN,10.16,NaN,Aguascalientes
197409,2018-10-16 01:00:00,9.55,0.51,9.06,7.30,NaN,10.60,NaN,Aguascalientes
197410,2018-10-16 02:00:00,10.32,0.55,8.77,7.32,NaN,-1.63,NaN,Aguascalientes


# CLEANING DATASET
In this section, cleaning and validation criteria are appied to the entire dataset.


SETTING AS NULL VALUES THOSE BELOW NEGATIVE DETECTION LIMIT

Data between the negative and positive detection limit values are replaced by half the detection limit value.

In [ ]:
s = '''
select
    time,

    CASE when
        (
            (O3 > -0.03) or (O3 is NULL)
        )
        then O3 else NULL
        END as O3,

    CASE when
        (
            (SO2 > -0.5) or (SO2 is NULL)
        )
        then SO2 else NULL
        END as SO2,

    CASE when
        (
            (NO2 > -0.4) or (NO2 is NULL)
        )
        then NO2 else NULL
        END as NO2,

    CASE when
        (
            (NO > -0.4) or (NO is NULL)
        )
        then NO else NULL
        END as NO,

    CASE when
        (
            (CO > -0.04) or (CO is NULL)
        )
        then CO else NULL
        END as CO,

    CASE when
        (
            (PM10 > -4.0) or (PM10 is NULL)
        )
        then PM10 else NULL
        END as PM10,

    CASE when
        (
            (PM25 > -4.0) or (PM25 is NULL)
        )
        then PM25 else NULL
        END as PM25,

    region

from pollution_raw
'''
pollution_clean = connection.execute(s).df()

REMOVING EXTRAORDINARY HIGH VALUES ABOVE SPAN LIMITS

In [ ]:
s = '''
select
    time,

    CASE when
        (
            (O3 > 0.03) or (O3 is NULL)
        )
        then O3 else 0.015
        END as O3,

    CASE when
        (
            (SO2 > 0.5) or (SO2 is NULL)
        )
        then SO2 else 0.25
        END as SO2,

    CASE when
        (
            (NO2 > 0.4) or (NO2 is NULL)
        )
        then NO2 else 0.2
        END as NO2,

    CASE when
        (
            (NO > 0.4) or (NO is NULL)
        )
        then NO else 0.2
        END as NO,

    CASE when
        (
            (CO > 0.04) or (CO is NULL)
        )
        then CO else 0.02
        END as CO,

    CASE when
        (
            (PM10 > 4.0) or (PM10 is NULL)
        )
        then PM10 else 2.0
        END as PM10,

    CASE when
        (
            (PM25 > 4.0) or (PM25 is NULL)
        )
        then PM25 else 2.0
        END as PM25,

    region

from pollution_clean
'''
pollution_clean = connection.execute(s).df()


SORTING DATA BY DATE

Data is finally sorted by date. Pay attention to regions sice timestamp will repeat dates for each region. If you want to save this dataframe as csv file, then you should run the last section SAVING CLEANED DATASET AS CSV.

In [ ]:
pollution_clean = pollution_clean.sort_values(by = 'Time')

pollution_clean



,Time,O3,SO2,NO2,NO,CO,PM10,PM25,region
164192,2015-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aguascalientes
164193,2015-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aguascalientes
164194,2015-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aguascalientes
164195,2015-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aguascalientes
164196,2015-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aguascalientes
...,...,...,...,...,...,...,...,...,...
164189,2024-11-11 01:00:00,NaN,0.74,NaN,0.2,4.64,8.86,7.81,Saltillo
164190,2024-11-11 02:00:00,0.015,0.82,NaN,0.2,4.88,18.85,15.71,Saltillo
81362,2024-11-11 02:00:00,NaN,NaN,NaN,NaN,NaN,15.21,NaN,Morelia
81363,2024-11-11 03:00:00,NaN,NaN,NaN,NaN,NaN,16.92,NaN,Morelia


#Z-SCORE EVALUATION FOR OUTLIER DETECTION

 Z-score thresholds can be modified as necesary. Default values were defined for long data series (more than five years). Thresholds may drecrease as the specific period does.

In [ ]:
dft = pollution_clean.groupby('region').agg(o3_avg=('O3','mean'),
                            o3_std=('O3', 'std'),
                            so2_avg=('SO2','mean'),
                            so2_std=('SO2', 'std'),
                            no2_avg=('NO2','mean'),
                            no2_std=('NO2', 'std'),
                            no_avg=('NO','mean'),
                            no_std=('NO', 'std'),
                            co_avg=('CO','mean'),
                            co_std=('CO', 'std'),
                            pm10_avg=('PM10','mean'),
                            pm10_std=('PM10', 'std'),
                            pm25_avg=('PM25','mean'),
                            pm25_std=('PM25', 'std'),
                            ).reset_index()

pollution_clean = connection.execute('''
with qz as (select *,
    (pollution_clean.O3 - dft.o3_avg)/dft.o3_std as o3_zvalue,
    (pollution_clean.SO2 - dft.so2_avg)/dft.so2_std as so2_zvalue,
    (pollution_clean.NO2 - dft.no2_avg)/dft.no2_std as no2_zvalue,
    (pollution_clean.NO - dft.no_avg)/dft.no_std as no_zvalue,
    (pollution_clean.CO - dft.co_avg)/dft.co_std as co_zvalue,
    (pollution_clean.PM10 - dft.pm10_avg)/dft.pm10_std as pm10_zvalue,
    (pollution_clean.PM25 - dft.pm25_avg)/dft.pm25_std as pm25_zvalue,


    from pollution_clean

    left join dft on dft.region = pollution_clean.region)

select
    Time,
    region,
    CASE when
        (
            (o3_zvalue between -4 and 4) or (O3 is NULL)
        )
        then O3 else NULL
    END as O3,

    CASE when
        (
            (so2_zvalue between -4 and 4) or (SO2 is NULL)
        )
        then SO2 else NULL
    END as SO2,

    CASE when
        (
            (no2_zvalue between -6 and 6) or (NO2 is NULL)
        )
        then NO2 else NULL
    END as NO2,

    CASE when
        (
            (no_zvalue between -4 and 4) or (NO is NULL)
        )
        then NO else NULL
    END as NO,

    CASE when
        (
            (co_zvalue between -4 and 4) or (CO is NULL)
        )
        then CO else NULL
    END as CO,

    CASE when
        (
            (pm10_zvalue between -6 and 6) or (PM10 is NULL)
        )
        then PM10 else NULL
    END as PM10,

    CASE when
        (
            (pm25_zvalue between -6 and 6) or (PM25 is NULL)
        )
        then PM25 else NULL
    END as PM25



from qz
''').df()

pollution_clean.describe()

pollution_clean

,Time,region,O3,SO2,NO2,NO,CO,PM10,PM25
0,2015-01-01 00:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-01 01:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-01 02:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-01 03:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-01 04:00:00,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
197407,2019-06-11 15:00:00,Morelia,NaN,1.51,NaN,NaN,0.22,15.95,NaN
197408,2019-06-11 16:00:00,Saltillo,48.79,NaN,0.20,0.2,NaN,9.73,2.00
197409,2019-06-11 16:00:00,Morelia,NaN,0.58,NaN,NaN,0.20,12.62,NaN
197410,2019-06-11 17:00:00,Saltillo,64.92,NaN,6.19,0.5,NaN,16.18,12.04


#SPLIT DATA BY REGION

Uncomment the line for the region you want.

In [ ]:
pollution_clean = pollution_clean.loc[pollution_clean['region']=='Morelia']
# pollution_clean = pollution_clean.loc[pollution_clean['region']=='Saltillo']
# pollution_clean = pollution_clean.loc[pollution_clean['region']=='Aguascalientes']

pollution_clean


,Time,region,O3,SO2,NO2,NO,CO,PM10,PM25
2049,2019-06-11 18:00:00,Morelia,NaN,0.60,NaN,NaN,0.22,14.33,NaN
2051,2019-06-11 19:00:00,Morelia,NaN,1.15,NaN,NaN,0.29,18.05,NaN
2052,2019-06-11 20:00:00,Morelia,NaN,0.96,NaN,NaN,0.36,28.60,NaN
2054,2019-06-11 21:00:00,Morelia,NaN,1.12,NaN,NaN,0.33,32.90,NaN
2056,2019-06-11 22:00:00,Morelia,NaN,0.97,NaN,NaN,0.30,22.55,NaN
...,...,...,...,...,...,...,...,...,...
197402,2019-06-11 13:00:00,Morelia,NaN,2.32,NaN,NaN,0.23,20.29,NaN
197405,2019-06-11 14:00:00,Morelia,NaN,2.01,NaN,NaN,0.23,13.17,NaN
197407,2019-06-11 15:00:00,Morelia,NaN,1.51,NaN,NaN,0.22,15.95,NaN
197409,2019-06-11 16:00:00,Morelia,NaN,0.58,NaN,NaN,0.20,12.62,NaN


# SELECT SPECIFIC TIME PERIOD

insert speficfic start and end dates including time information following the format YYY-mm-dd HH:MM:SS

In [ ]:
start_date ='2016-01-01 00:00:00'
end_date = '2023-12-31 23:00:00'

period = (pollution_clean['Time'] >= start_date) & (pollution_clean['Time'] <= end_date)

pollution_clean = pollution_clean.loc[period]

pollution_clean

,Time,region,O3,SO2,NO2,NO,CO,PM10,PM25
2049,2019-06-11 18:00:00,Morelia,NaN,0.60,NaN,NaN,0.22,14.33,NaN
2051,2019-06-11 19:00:00,Morelia,NaN,1.15,NaN,NaN,0.29,18.05,NaN
2052,2019-06-11 20:00:00,Morelia,NaN,0.96,NaN,NaN,0.36,28.60,NaN
2054,2019-06-11 21:00:00,Morelia,NaN,1.12,NaN,NaN,0.33,32.90,NaN
2056,2019-06-11 22:00:00,Morelia,NaN,0.97,NaN,NaN,0.30,22.55,NaN
...,...,...,...,...,...,...,...,...,...
197402,2019-06-11 13:00:00,Morelia,NaN,2.32,NaN,NaN,0.23,20.29,NaN
197405,2019-06-11 14:00:00,Morelia,NaN,2.01,NaN,NaN,0.23,13.17,NaN
197407,2019-06-11 15:00:00,Morelia,NaN,1.51,NaN,NaN,0.22,15.95,NaN
197409,2019-06-11 16:00:00,Morelia,NaN,0.58,NaN,NaN,0.20,12.62,NaN


# DALY AVERAGES AND DIURNAL DATA

In [ ]:
'24h average'

pollution_clean.loc[:,'Date'] = pollution_clean.Time.dt.date   # Create column with dates only

def calculate_daily_mean(group, is_circular=False):   # is_circular conditional for circular variables such as wind direction. To be declared on function custom_mean.
  valid_count = group.notna().sum()
  if valid_count >= 18:
    if is_circular:
      rad = np.deg2rad(group)
      mean_sin = np.mean(np.sin(rad))
      mean_cos = np.mean(np.cos(rad))
      mean_angle = np.arctan2(mean_sin, mean_cos)
      return np.rad2deg(mean_angle) % 360     # Ensure that mean_angle is between 0 an 360
    else:
      return group.mean().round(2)
  else:
    return np.nan

def custom_mean(group):
  return {
      'O3': calculate_daily_mean(group['O3']),
      'SO2': calculate_daily_mean(group['SO2']),
      'NO2': calculate_daily_mean(group['NO2']),
      'NO': calculate_daily_mean(group['NO']),
      'CO': calculate_daily_mean(group['CO']),
      'PM10': calculate_daily_mean(group['PM10']),
      'PM25': calculate_daily_mean(group['PM25']),
  }

daily_mean = pollution_clean.groupby(['region', 'Date']).apply(custom_mean).apply(pd.Series)

print(daily_mean)

"hourly average (output: 24 lines) diurnal cycles taking the median for better representation"
# pollution_clean = pollution_clean.groupby(['region', pollution_clean['Time'].dt.hour]).median()

pollution_clean



<ipython-input-19-914fd8a795d9>:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily_mean = pollution_clean.groupby(['region', 'Date']).apply(custom_mean).apply(pd.Series)


                       O3   SO2   NO2    NO    CO   PM10   PM25
region  Date                                                   
Morelia 2016-01-01  36.55  0.47  3.42  0.37  0.79  30.26  20.17
        2016-01-02  24.67  0.32  4.13  1.30  0.78  21.82  12.98
        2016-01-03  26.57  0.28  3.06  0.57  0.73  15.21   9.19
        2016-01-04  21.30  0.27  7.49  1.11  0.78  24.61  13.93
        2016-01-05  22.22  0.31  3.59  1.06  0.76  17.22  10.27
...                   ...   ...   ...   ...   ...    ...    ...
        2023-12-27  22.76   NaN  4.92  1.35  0.30   5.82   4.78
        2023-12-28  23.33   NaN  4.58  0.82  0.35   8.81   6.58
        2023-12-29  27.86   NaN  3.33  0.27  0.38  11.42   9.05
        2023-12-30  23.63   NaN  6.82  1.35  0.42  18.06  13.33
        2023-12-31    NaN   NaN   NaN   NaN   NaN    NaN    NaN

[2922 rows x 7 columns]


,Time,region,O3,SO2,NO2,NO,CO,PM10,PM25,Date
3672,2016-01-01 00:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-01-01
3673,2016-01-01 01:00:00,Morelia,26.82,0.25,2.15,0.2,0.72,24.18,12.86,2016-01-01
3674,2016-01-01 02:00:00,Morelia,28.94,0.25,2.25,0.2,0.73,25.27,17.26,2016-01-01
3675,2016-01-01 03:00:00,Morelia,26.58,0.25,2.48,0.2,0.74,25.16,21.16,2016-01-01
3676,2016-01-01 04:00:00,Morelia,23.99,0.25,3.42,0.2,0.80,32.90,23.58,2016-01-01
...,...,...,...,...,...,...,...,...,...,...
73795,2023-12-31 19:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31
73796,2023-12-31 20:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31
73797,2023-12-31 21:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31
73798,2023-12-31 22:00:00,Morelia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-12-31


#SAVING CLEANED DATASET AS CSV FILE

In [ ]:
date_now = datetime.datetime.now() - datetime.timedelta(hours=6)
date_now = date_now.strftime('%Y-%m-%d %H:%M')

file_name = 'pollution_clean_' + str(date_now) + '.csv'

pollution_clean.to_csv('/content/drive/MyDrive/ATPAD_COLAB/OUTPUT_FILES/'+ file_name)

daily_mean.to_csv('/content/drive/MyDrive/ATPAD_COLAB/OUTPUT_FILES/daily_mean_pollution_' + str(date_now) + '.csv')